In [7]:
import pandas as pd

air_quality_df = pd.read_csv('data/datathon24_parte1/air_quality_gijon.csv')
meteo_df = pd.read_csv('data/datathon24_parte1/meteo_gijon.csv')
mobility_df = pd.read_csv('data/datathon24_parte1/movility_gijon.csv')

selected_columns = [
    'date',  # Include the date for merging
    'SO2_GijonGlobal', 'NO_GijonGlobal', 'NO2_GijonGlobal', 
    'CO_GijonGlobal', 'PM10_GijonGlobal', 'O3_GijonGlobal'
]
filtered_air_quality_df = air_quality_df[selected_columns]

merged_df = filtered_air_quality_df.merge(meteo_df, on='date', how='outer').merge(mobility_df, on='date', how='outer')

prediction_df = pd.read_csv('data/datathon24_parte2/prediction.csv')

merged_df = merged_df.merge(prediction_df, on='date', how='outer')


In [9]:
merged_df.to_csv('data/datathon24_parte2/merged_data_partII.csv', index=False)

In [16]:
# Convert 'date' column to datetime and sort by date
merged_df['date'] = pd.to_datetime(merged_df['date'])
merged_df.sort_values('date', inplace=True)

# Create date-related features
merged_df['day_of_week'] = merged_df['date'].dt.dayofweek
merged_df['month'] = merged_df['date'].dt.month

# Choose a rolling window size, e.g., 7 days
window_size = 7

# Calculate rolling features for air quality measurements
for col in ['SO2_GijonGlobal', 'NO_GijonGlobal', 'NO2_GijonGlobal', 'CO_GijonGlobal', 'PM10_GijonGlobal', 'O3_GijonGlobal']:
    merged_df[f'{col}_rolling_mean'] = merged_df[col].rolling(window=window_size, min_periods=1).mean()
    merged_df[f'{col}_rolling_std'] = merged_df[col].rolling(window=window_size, min_periods=1).std()

# Create lag features, e.g., for the previous day
for col in ['SO2_GijonGlobal', 'NO_GijonGlobal', 'NO2_GijonGlobal', 'CO_GijonGlobal', 'PM10_GijonGlobal', 'O3_GijonGlobal']:
    merged_df[f'{col}_lag1'] = merged_df[col].shift(1)

# Fill missing values (forward-fill for time-series data)
merged_df.fillna(method='ffill', inplace=True)

# Normalize the continuous features
from sklearn.preprocessing import StandardScaler

# Exclude non-continuous or non-relevant features for scaling
non_scaled_cols = ['date', 'ICA_PM25_target', 'day_of_week', 'month']
features_to_scale = [col for col in merged_df.columns if col not in non_scaled_cols]

# Apply standardization
scaler = StandardScaler()
merged_df[features_to_scale] = scaler.fit_transform(merged_df[features_to_scale])

# Separate the prediction period (December 2023) data
x_final_prediction = merged_df[(merged_df['date'] >= '2023-12-01') & (merged_df['date'] <= '2023-12-31')].drop(columns=['ICA_PM25_target'])

# Drop the date column and other non-predictive columns for modeling
X_train = merged_df[merged_df['date'] < '2023-12-01'].drop(columns=['date', 'ICA_PM25_target'])
y_train = merged_df[merged_df['date'] < '2023-12-01']['ICA_PM25_target']

# Make sure the indices are reset for the training data
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
x_final_prediction.reset_index(drop=True, inplace=True)

In [18]:
# Handle any remaining missing values
X_train = X_train.fillna(method='bfill', axis=0)  # Backward-fill

# Check for multicollinearity (example using VIF)
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["feature"] = X_train.columns
vif_data["VIF"] = [variance_inflation_factor(X_train.values, i) for i in range(len(X_train.columns))]

# Print the features with high VIF values
print(vif_data[vif_data['VIF'] > 10])

# Feature selection: drop features based on domain knowledge, VIF, or preliminary feature importance
# For example, to drop features with VIF greater than 10
high_vif_features = vif_data[vif_data['VIF'] > 10]["feature"].tolist()
X_train.drop(columns=high_vif_features, inplace=True)

# Check class balance
class_counts = y_train.value_counts()
print(class_counts)

                                feature        VIF
16         C01_BibioArenaMestas_trafico  12.882529
20             C05_CotoViesques_trafico  16.501066
21             C06_LlanoPumarin_trafico  10.112198
23          C08_NuevoGijonBrana_trafico  24.136407
25  C10_ContruecesGijonSurRoces_trafico  11.758029
26   C11_NatahoyoMoredaTremañes_trafico  16.172234
27                   C12_Ceares_trafico  41.422942
32          NO_GijonGlobal_rolling_mean  12.286280
34         NO2_GijonGlobal_rolling_mean  10.073702
ICA_PM25_target
0.0    1342
1.0     453
Name: count, dtype: int64
